## 数据处理

In [11]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests
from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [12]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1209《欢迎来到地球》测试1群.txt"

# 设定时间范围
start_time = "2025-12-03 14:00:00"
end_time   = "2025-12-03 14:30:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



195 lines
{"发言日期": "2025-12-03", "发言时间": "14:00:00", "玩家ID": "何方路人甲(978617207)", "玩家消息": "我第一次玩就是跟怪诞虫和残肢"}


## 大模型分类

## 定义

In [13]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from testimport import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,build_user_prompt_classify,build_user_prompt_cluster_correct,call_ark_chat_completions,
    extract_valid_json_lines,jsonl_to_dataframe_with_intent,
    load_whitelist,extract_clusters_from_output, update_and_save_whitelist,attach_cluster_to_messages,
    parse_clusters_from_output,attach_cluster_to_batch,build_class_map_from_text,
    create_intent_excel_styled,append_json_to_excel_by_cat_and_tag,postprocess_excel_by_topic
    
)

## 设置参数

In [14]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点

PROMPT_MD_PATH01 = Path("提示词1.md") # 模型#1 system 提示词（筛相关）
PROMPT_MD_PATH02 = Path("提示词2.md") # 模型#2 system 提示词（做分类）
PROMPT_MD_PATH03 = Path("2话题分类.md") # 模型#2 system 提示词（分话题）
PROMPT_MD_PATH04 = Path("test话提簇校正.md") # 模型#3 system 提示词（日聚合）
EXCEL_FILE       = Path("test群.xlsx") 
BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 开始运行

In [15]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)


create_intent_excel_styled(EXCEL_FILE)



# 植入白名单 #
# --- 白名单初始化 ---
WHITE_LIST_PATH = Path("话题簇白名单.jsonl")
if not WHITE_LIST_PATH.exists():
    WHITE_LIST_PATH.write_text("", encoding="utf-8")  # 自动创建空文件
whitelist = load_whitelist(WHITE_LIST_PATH)  # 首批为空或加载已有


total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        
        # --- 模型 #2：分类 ---
        user_prompt2 = build_user_prompt_classify(output_filter)
        output_class = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        print(output_class)
        if not output_class:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        # --- 模型 #3：话题簇划分 ---
        user_prompt3 = build_user_prompt_clsuter(output_class)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt03,
            user_prompt=user_prompt3,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
      # --- 模型 #4：话提簇校正 ---
        user_prompt4 = build_user_prompt_cluster_correct(output_cluster, whitelist)
        output_classify = call_ark_chat_completions(
        api_url=API_URL,
        api_key=API_KEY,
        model=V3_MODEL_ID,
        system_prompt=system_prompt04,  # 模型#4 提示词
        user_prompt=user_prompt4,
        temperature=TEMPERATURE,
        max_tokens=MAX_TOKENS,
        timeout=TIMEOUT_SEC,
        retries=RETRIES,
        )
        
        # ② 白名单更新
        try:
            new_clusters = extract_clusters_from_output(output_classify)
            if new_clusters:
                whitelist = update_and_save_whitelist(
                    WHITE_LIST_PATH, whitelist, new_clusters
                )
            else:
                tqdm.write(f"[批次 {b+1}] ⚪ 本批次无新增话题簇。")
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠️ 白名单更新出错：{e}")

        cls_map = build_class_map_from_text(output_class)
        cluster_list = parse_clusters_from_output(output_classify)
        if not cluster_list:
            tqdm.write(f"[批次 {b+1}] ⚠️ parse_clusters_from_output 解析结果为空，本批次跳过。")
            continue
        
        final_records = attach_cluster_to_batch(
            cluster_list=cluster_list,
            batch_raw_lines=batch_lines,
            cls_map=cls_map,
        )


        append_json_to_excel_by_cat_and_tag(final_records, EXCEL_FILE)

        batch_written = len(final_records)
        written_total += batch_written
        tqdm.write(f"[批次 {b+1}] ✅ 写入 {batch_written} 条。")
        
  
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)
postprocess_excel_by_topic(EXCEL_FILE, gap_minutes=60, nat_policy="skip")

print("\n✅ 全部批次处理完成！")
print(f"输入总数：{total}")
print(f"写入总数：{written_total}")


✅ 创建并套样式：test群.xlsx
准备处理 195 条，共 1 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                       | 0/1 [00:00<?, ?批/s]

{"发言日期": "2025-12-03", "发言时间": "14:00:00", "玩家ID": "何方路人甲(978617207)", "玩家消息": "我第一次玩就是跟怪诞虫和残肢", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:07", "玩家ID": "千墓流沙(919072537)", "玩家消息": "启动！", "分类标签": 4}
{"发言日期": "2025-12-03", "发言时间": "14:00:09", "玩家ID": "何方路人甲(978617207)", "玩家消息": "那时候还不会摆道具", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:15", "玩家ID": "何方路人甲(978617207)", "玩家消息": "一直疯", "分类标签": 4}
{"发言日期": "2025-12-03", "发言时间": "14:00:17", "玩家ID": "枫火(276465118)", "玩家消息": "又是玩球", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:30", "玩家ID": "甲乙丙丁(449511)", "玩家消息": "又是西洋剑", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:39", "玩家ID": "甲乙丙丁(449511)", "玩家消息": "[图片]", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:48", "玩家ID": "空(3316920376)", "玩家消息": "啊要重新下载", "分类标签": 5}
{"发言日期": "2025-12-03", "发言时间": "14:00:49", "玩家ID": "尘(3411480843)", "玩家消息": "[图片]", "分类标签": 1}
{"发言日期": "2025-12-03", "发言时间": "14:00:53", "玩家ID": "尘(3411480843)", "玩家消息": "开PVP了", "分类标签": 1}
{"发言日期": "2025-12-03", "

🔥 批处理进度: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [05:12<00:00, 312.39s/批]      

⚪ 无新增话题簇
[批次 1] ⚠️ parse_clusters_from_output 解析结果为空，本批次跳过。


AttributeError: 'MergedCell' object attribute 'value' is read-only

In [7]:
print(final_records)

[{'发言日期': '2025-12-03', '发言时间': '14:00:00', '玩家ID': '何方路人甲(978617207)', '玩家消息': '我第一次玩就是跟怪诞虫和残肢', '一级分类': 1, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:07', '玩家ID': '千墓流沙(919072537)', '玩家消息': '启动！', '一级分类': None, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:09', '玩家ID': '何方路人甲(978617207)', '玩家消息': '那时候还不会摆道具', '一级分类': 1, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:15', '玩家ID': '何方路人甲(978617207)', '玩家消息': '一直疯', '一级分类': None, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:17', '玩家ID': '璃奈(511808399)', '玩家消息': 'go', '一级分类': None, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:17', '玩家ID': '枫火(276465118)', '玩家消息': '又是玩球', '一级分类': 1, '话题簇': '怪诞虫开局体验'}, {'发言日期': '2025-12-03', '发言时间': '14:00:30', '玩家ID': '甲乙丙丁(449511)', '玩家消息': '又是西洋剑', '一级分类': 1, '话题簇': '台球开局讨论'}, {'发言日期': '2025-12-03', '发言时间': '14:00:39', '玩家ID': '甲乙丙丁(449511)', '玩家消息': '[图片]', '一级分类': 1, '话题簇': '西洋剑开局讨论'}, {'发言日期': '2025-12-03', '发言时间': '14:00:48', '玩家ID': '空(3316